In [14]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd


json_keyfile = "/Users/tanishqa/Downloads/vader-sentiment-analysis-e6711036e61c.json"

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

creds = ServiceAccountCredentials.from_json_keyfile_name(json_keyfile, scope)

client = gspread.authorize(creds)
sheet = client.open("Copy of Spreadsheet Data w/ Sentiment Predictions").sheet1
data = sheet.get_all_records()
df = pd.DataFrame(data)

print(df.head())  


  Column1       Dates                                              Links  \
0          2023-04-25  https://www.reddit.com/r/wallstreetbets/commen...   
1       1  2023-01-24  https://www.reddit.com/r/wallstreetbets/commen...   
2       1  2023-10-29  https://www.reddit.com/r/wallstreetbets/commen...   
3       8  2023-07-25  https://www.reddit.com/r/wallstreetbets/commen...   
4       2  2023-04-27  https://www.reddit.com/r/wallstreetbets/commen...   

                                                text     label  
0  Bro bro brosuf bitch as mother fucking regard ...  0.010114  
1  FUCKING MORONS!!! MSFT Misses and it goes up. ...   0.01012  
2  Seriously I don’t know why any sane company wo...  0.010152  
3  Where is $30k $MSFT calls guy? I want kick his...  0.010177  
4  Ugh I’m a fucking dumbass\n\n50 aapl 170 calls...  0.010268  


In [15]:
print(df.columns)

Index(['Column1', 'Dates', 'Links', 'text', 'label'], dtype='object')


In [16]:
from transformers import DistilBertTokenizer
from sklearn.model_selection import train_test_split


tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")


def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)


df_tokenized = df.apply(lambda row: tokenizer(row['text'], padding="max_length", truncation=True), axis=1)

#Split training data into 80% train 20% test
train_data, test_data = train_test_split(df, test_size=0.2)


print(train_data.head())


      Column1       Dates                                              Links  \
5780        1  2023-07-27  https://www.reddit.com/r/wallstreetbets/commen...   
165         2  2023-10-10  https://www.reddit.com/r/wallstreetbets/commen...   
26224       8  2023-06-22  https://www.reddit.com/r/wallstreetbets/commen...   
15171       1  2023-07-26  https://www.reddit.com/r/wallstreetbets/commen...   
11119       5  2023-07-25  https://www.reddit.com/r/wallstreetbets/commen...   

                                                    text     label  
5780   JFC how is MSFT red on a nearly 2% QQQ day. Bi...  0.107739  
165       lets go msft punch these fuckers in the \nface  0.011211  
26224  Heh, so market green today thank to 3 companie...  0.990135  
15171                                #MSFT CAT5 TURDNADO  0.525544  
11119  TSLA going to collapse back to $200. MSFT back...  0.451523  


In [ ]:
import torch


train_encodings = tokenizer(list(train_data['text']), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(list(test_data['text']), truncation=True, padding=True, max_length=512)


train_labels = torch.tensor(train_data['label'].values)  
test_labels = torch.tensor(test_data['sentiment_column'].values)  


train_inputs = torch.tensor(train_encodings['input_ids'])
test_inputs = torch.tensor(test_encodings['input_ids'])


In [ ]:

print(train_data['label'].unique())  #Check non-numeric values


In [18]:

train_labels = torch.tensor(train_data['label'].values, dtype=torch.float)
test_labels = torch.tensor(test_data['label'].values, dtype=torch.float)


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [19]:

train_data['label'] = pd.to_numeric(train_data['label'], errors='coerce')
test_data['label'] = pd.to_numeric(test_data['label'], errors='coerce')


train_data = train_data.dropna(subset=['label'])
test_data = test_data.dropna(subset=['label'])

train_labels = torch.tensor(train_data['label'].values, dtype=torch.float)
test_labels = torch.tensor(test_data['label'].values, dtype=torch.float)


print(train_labels.dtype)
print(test_labels.dtype)


torch.float32
torch.float32


In [15]:
print(train_data['label'].head())


18047    0.576233
22994    0.844681
12403    0.492164
16658    0.547687
20689    0.683417
Name: label, dtype: float64


In [20]:
from transformers import DistilBertTokenizer


tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")


train_encodings = tokenizer(list(train_data['text']), truncation=True, padding=True, max_length=512, return_tensors="pt")
test_encodings = tokenizer(list(test_data['text']), truncation=True, padding=True, max_length=512, return_tensors="pt")



In [21]:

train_inputs = torch.tensor(train_encodings['input_ids'])
test_inputs = torch.tensor(test_encodings['input_ids'])


/var/folders/t4/hm0qvm6d4y5cnzj9d_ll99lc0000gn/T/ipykernel_64247/2625499050.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_inputs = torch.tensor(train_encodings['input_ids'])
/var/folders/t4/hm0qvm6d4y5cnzj9d_ll99lc0000gn/T/ipykernel_64247/2625499050.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_inputs = torch.tensor(test_encodings['input_ids'])


In [23]:
import torch
from torch.utils.data import Dataset

class SentimentDataset(Dataset):
    def __init__(self, inputs, labels, attention_mask):
        self.inputs = inputs
        self.labels = labels
        self.attention_mask = attention_mask

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }


train_dataset = SentimentDataset(train_encodings['input_ids'], train_labels, train_encodings['attention_mask'])
test_dataset = SentimentDataset(test_encodings['input_ids'], test_labels, test_encodings['attention_mask'])


In [24]:
from torch.utils.data import DataLoader

#Create dataloader for batching
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16)



In [25]:
from transformers import DistilBertForSequenceClassification


model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=1)  # num_labels=1 for regression


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
device = torch.device("cpu")  
model.to(device)



DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
import torch.optim as optim
from torch import nn
import torch.optim as optim
optimizer = optim.AdamW(model.parameters(), lr=5e-5)



loss_fn = torch.nn.MSELoss()  # Use MSELoss 


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


epochs = 1
for epoch in range(epochs):
    print(f"Starting epoch {epoch+1}...")  
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        optimizer.zero_grad()

        
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels'] 

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        
        loss.backward()
        optimizer.step()

    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(train_dataloader)}")

Starting epoch 1...
